In [ ]:
!pip install pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas_datareader.data as web
import datetime

In [ ]:
import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as wb

In [ ]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# start = datetime.datetime(2019,9,15)
# end = datetime.datetime(2021,9,15)

In [ ]:
!pip install PyPortfolioOpt

In [ ]:
def get_stock(ticker):
    data = wb.DataReader(f"{ticker}",'2023-1-1')
    data[f'{ticker}'] = data["Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 

In [ ]:
from functools import reduce

def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    print(df_merged.head())
    return df_merged


In [ ]:
stocks = ["^N225", "^GDAXI", "^GSPC", "^NDX", 
          "000001.SS","^SPGSCLP", "^SPGSGCP", "HYG", "TLT", 
          "XBT-USD"]
# stocks = ["^N225", "^GDAXI", "^GSPC", "^NDX", 
#           "000001.SS", "HYG", "TLT"]
# stocks = ["^N225", "^GDAXI", "^GSPC", "^NDX", 
#           "000001.SS"]
portfolio = combine_stocks(stocks)

[*********************100%***********************]  1 of 1 completed
                   ^N225
Date                    
2023-01-04  25716.859375
2023-01-05  25820.800781
2023-01-06  25973.849609
2023-01-10  26175.560547
2023-01-11  26446.000000
[*********************100%***********************]  1 of 1 completed
                  ^GDAXI
Date                    
2023-01-02  14069.259766
2023-01-03  14181.669922
2023-01-04  14490.780273
2023-01-05  14436.309570
2023-01-06  14610.019531
[*********************100%***********************]  1 of 1 completed
                  ^GSPC
Date                   
2023-01-03  3824.139893
2023-01-04  3852.969971
2023-01-05  3808.100098
2023-01-06  3895.080078
2023-01-09  3892.090088
[*********************100%***********************]  1 of 1 completed
                    ^NDX
Date                    
2023-01-03  10862.639648
2023-01-04  10914.799805
2023-01-05  10741.219727
2023-01-06  11040.349609
2023-01-09  11108.450195
[*********************100%*****

In [ ]:
portfolio.to_csv("portfolio.csv", index=False)

In [ ]:
portfolio = pd.read_csv("portfolio.csv")

In [ ]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage


mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))

{'^N225': 0.79006, '^GDAXI': 0.06607, '^GSPC': 0.0, '^NDX': 0.12619, '000001.SS': 0.01037, '^SPGSCLP': 0.0, '^SPGSGCP': 0.0, 'HYG': 0.0, 'TLT': 0.0, 'XBT-USD': 0.00731}


In [ ]:
ef.portfolio_performance(verbose=True)

Expected annual return: 36.6%
Annual volatility: 12.3%
Sharpe Ratio: 2.82


(0.3655025165561391, 0.12272461280633837, 2.8152667069428707)

In [ ]:
from pypfopt import HRPOpt

In [ ]:
returns = portfolio.pct_change().dropna()

In [ ]:
hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()

In [ ]:
hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

In [ ]:
from pypfopt.efficient_frontier import EfficientCVaR

In [ ]:
S = portfolio.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()

cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

In [ ]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [ ]:
latest_prices = get_latest_prices(portfolio)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da.greedy_portfolio()

In [ ]:
da_cvar = DiscreteAllocation(cvar_weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da_cvar.greedy_portfolio()
print("Discrete allocation (CVAR):", allocation)
print("Funds remaining (CVAR): ${:.2f}".format(leftover))

Discrete allocation (CVAR): {'^NDX': 7, '^GSPC': 1, 'TLT': 1}
Funds remaining (CVAR): $2944.95


/usr/local/lib/python3.10/dist-packages/pypfopt/discrete_allocation.py:212: RuntimeWarning: invalid value encountered in true_divide
  current_weights /= current_weights.sum()
